In [8]:
import torch
print("Hello azue ML")

Hello azue ML


To create a simple neural network in PyTorch for sentence classification (valid grammar or not), we'll need to follow a few key steps:

1. Prepare the data: Create a dataset with sentences labeled as "valid" or "invalid."
2. Tokenize the sentences: Convert sentences into numerical representations (e.g., using tokenization and embedding).
3. Create the model: Define a simple neural network architecture.
4. Train the model: Use the training data to optimize the model.
5. Evaluate the model: Test the model's performance on a test set.

# 1. Prepare dataset

In [9]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import string

# Toy dataset with sentences and labels (1 for valid, 0 for invalid)
data = [
    ("The cat sat on the mat", 1),
    ("This is a valid sentence", 1),
    ("Sentence incorrect word", 0),
    ("Invalid sentence test word", 0),
    ("The quick brown fox jumps over", 0),
    ("A valid grammatical sentence", 1),
    ("Me very good at this", 0)
]

# Preprocessing the sentences by tokenizing and converting to lowercase
def preprocess(sentence):
    return sentence.translate(str.maketrans('', '', string.punctuation)).lower().split()

# Create a vocabulary
all_words = [word for sentence, _ in data for word in preprocess(sentence)]
vocab = {word: idx + 1 for idx, (word, _) in enumerate(Counter(all_words).items())}
vocab['<PAD>'] = 0  # Padding token

# Define a custom dataset for loading
class SentenceDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence, label = self.data[idx]
        tokenized_sentence = preprocess(sentence)
        # Convert tokens to indices
        sentence_indices = [self.vocab.get(word, 0) for word in tokenized_sentence]
        return torch.tensor(sentence_indices), torch.tensor(label)

# Custom collate function for padding
def collate_fn(batch):
    sentences, labels = zip(*batch)
    # Pad sentences to the maximum length in the batch
    sentences_padded = pad_sequence(sentences, batch_first=True, padding_value=0)
    labels = torch.tensor(labels)
    return sentences_padded, labels

dataset = SentenceDataset(data, vocab)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)


# 2. Create the model

In [10]:
class SentenceClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SentenceClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, (hn, _) = self.rnn(x)
        out = self.fc(hn[-1])
        return torch.sigmoid(out)

# Define model parameters
vocab_size = len(vocab)
embedding_dim = 8
hidden_dim = 16
output_dim = 1

model = SentenceClassifier(vocab_size, embedding_dim, hidden_dim, output_dim)


# 3. Train the model

Now let's define the training loop to optimize the model.

In [11]:
# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for sentences, labels in dataloader:
        # Pad sentences to have the same length
        sentences = torch.nn.functional.pad(sentences, (0, 10 - sentences.size(1)), value=0)  # Padding length 10
        labels = labels.float().view(-1, 1)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(sentences)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate accuracy
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}, Accuracy: {correct/total:.4f}")


Epoch [1/10], Loss: 0.6927, Accuracy: 0.7143
Epoch [2/10], Loss: 0.6930, Accuracy: 0.5714
Epoch [3/10], Loss: 0.6898, Accuracy: 0.5714
Epoch [4/10], Loss: 0.6898, Accuracy: 0.5714
Epoch [5/10], Loss: 0.6892, Accuracy: 0.5714
Epoch [6/10], Loss: 0.6924, Accuracy: 0.5714
Epoch [7/10], Loss: 0.6916, Accuracy: 0.5714
Epoch [8/10], Loss: 0.6850, Accuracy: 0.5714
Epoch [9/10], Loss: 0.6910, Accuracy: 0.5714
Epoch [10/10], Loss: 0.6910, Accuracy: 0.5714


# 4. Evaluate the Model

In [12]:
def evaluate(model, sentence, vocab):
    model.eval()
    sentence_indices = [vocab.get(word, 0) for word in preprocess(sentence)]
    sentence_tensor = torch.tensor(sentence_indices).unsqueeze(0)
    sentence_tensor = torch.nn.functional.pad(sentence_tensor, (0, 10 - sentence_tensor.size(1)), value=0)
    
    with torch.no_grad():
        output = model(sentence_tensor)
        prediction = (output > 0.5).item()
        return "Valid Grammar" if prediction == 1 else "Invalid Grammar"

# Test the model
test_sentence = "Me very good at this"
result = evaluate(model, test_sentence, vocab)
print(f"Sentence: '{test_sentence}' - {result}")


Sentence: 'Me very good at this' - Invalid Grammar
